In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from mindquantum.core.gates import X, Y, Z, S, H, RX, RY, RZ, Measure, UnivMathGate, DepolarizingChannel
from mindquantum.core.circuit import Circuit
from mindquantum.simulator import Simulator

In [34]:
def SpinChain(n,J,h):
    X = np.array([[0, 1], [1, 0]],dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]],dtype=complex)
    Z = np.array([[1, 0], [0, -1]],dtype=complex)
    def hopping(P,i):
        assert i < n, "i should be less than n"
        if i == 0 or i == n-1:
            matrix = P
        else:
            matrix = np.eye(2,dtype=complex)
        for j in range(1,n):
            if j == i or j == i+1:
                matrix = np.kron(P,matrix)
            else:
                matrix = np.kron(np.eye(2,dtype=complex),matrix)
        return matrix
    def potential(P,i):
        assert i < n, "i should be less than n"
        if i == 0:
            matrix = P
        else:
            matrix = np.eye(2,dtype=complex)
        for j in range(1,n):
            if j == i:
                matrix = np.kron(P,matrix)
            else:
                matrix = np.kron(np.eye(2,dtype=complex),matrix)
        return matrix
    
    # hopping term
    HoppingX = np.zeros((2**n,2**n),dtype=complex)
    HoppingY = np.zeros((2**n,2**n),dtype=complex)
    HoppingZ = np.zeros((2**n,2**n),dtype=complex)
    if n == 2:
        for i in range(n-1):
            HoppingX += hopping(X,i)*J[0]
            HoppingY += hopping(Y,i)*J[1]
            HoppingZ += hopping(Z,i)*J[2]
    else:
        for i in range(n):
            HoppingX += hopping(X,i)*J[0]
            HoppingY += hopping(Y,i)*J[1]
            HoppingZ += hopping(Z,i)*J[2]
    # potential term
    PotentialX = np.zeros((2**n,2**n),dtype=complex)
    PotentialY = np.zeros((2**n,2**n),dtype=complex)
    PotentialZ = np.zeros((2**n,2**n),dtype=complex)
    for i in range(n):
        PotentialX += potential(X,i)*h[0]
        PotentialY += potential(Y,i)*h[1]
        PotentialZ += potential(Z,i)*h[2]
    return HoppingX+HoppingY+HoppingZ+PotentialX+PotentialY+PotentialZ

def TimeEvolution(H,t):
    eigenv,U = np.linalg.eig(H)
    diag = np.diag(np.exp(-1.j*t*eigenv))
    return U@diag@np.linalg.inv(U)

def IPEA(m,hamil,ground_state,size = 1000, error = 0.0):
    n = int(np.log2(hamil.shape[0]))
    sim = Simulator('mqvector',n+1)
    target_bit = [i+1 for i in range(n)]

    phi = 0
    for i in tqdm(range(m-1,-1,-1)):
        Ut = TimeEvolution(hamil,2**i)
        randmat = np.random.randn(2**n,2**n)+1.j*np.random.randn(2**n,2**n)
        randmat = randmat*error*(2**i)
        Ut = Ut + randmat

        circ = Circuit()
        circ += H.on(0)
        evol_gate = UnivMathGate('Ut',Ut)
        circ += evol_gate.on(target_bit,0)
        Rz = np.array([[1.0, 0.0], [0.0, np.exp(1.j*phi)]],dtype=complex)
        circ += UnivMathGate('Rz',Rz).on(0)
        circ += H.on(0)
        circ += Measure('q0').on(0)
        
        sim.reset()
        sim.set_qs(np.kron(ground_state,[1.0,0.0]))
        result = sim.sampling(circuit=circ,shots=size)
        samples = result.data
        try:
            zero = samples['0']
        except:
            zero = 0
        try:
            one = samples['1']
        except:
            one = 0
        if i == 0:
            if zero > one:
                phi = -phi
            else:
                phi = -(phi - np.pi)
        else:
            if zero > one:
                phi = phi/2
            else:
                phi = phi/2 + np.pi/2
    return -phi

In [35]:
n = 8
J = [-1.0,-1.0,-1.5]
h = [1.5,0.0,0.5]
hamil = SpinChain(n,J,h)
ground_energy = np.linalg.eigvalsh(hamil)[0]

hamil = hamil*0.4 + np.identity(hamil.shape[0])*11
ground_state = np.linalg.eig(hamil)[1][:,0]

In [ ]:
dU_error = [1e-1,1e-2,1e-3,1e-4,1e-5,0]
samples = []
for i in tqdm(range(10),desc='samples',colour='blue'):
    energy_error = []
    for e in tqdm(dU_error,desc='error',colour='red'):
        estimate_phi = IPEA(m = 11, hamil = hamil, ground_state = ground_state, error = e)
        estimate_energy = (estimate_phi-11)/0.4
        energy_error.append(estimate_energy-ground_energy)
    samples.append(energy_error)

samples:   0%|          | 0/10 [00:00<?, ?it/s]











100%|██████████| 11/11 [00:06<00:00,  1.64it/s]












100%|██████████| 11/11 [00:05<00:00,  1.93it/s]












100%|██████████| 11/11 [00:05<00:00,  1.93it/s]












100%|██████████| 11/11 [00:08<00:00,  1.26it/s]












100%|██████████| 11/11 [00:05<00:00,  1.85it/s]












samples:  10%|█         | 1/10 [00:37<05:39, 37.72s/it]











100%|██████████| 11/11 [00:09<00:00,  1.22it/s]












100%|██████████| 11/11 [00:06<00:00,  1.61it/s]












100%|██████████| 11/11 [00:08<00:00,  1.29it/s]












100%|██████████| 11/11 [00:07<00:00,  1.42it/s]












100%|██████████| 11/11 [00:07<00:00,  1.46it/s]












samples:  20%|██        | 2/10 [01:24<05:45, 43.19s/it]











100%|██████████| 11/11 [00:05<00:00,  1.90it/s]












100%|██████████| 11/11 [00:08<00:00,  1.29it/s]












100%|██████████| 11/11 [00:07<00:00,  1.56it/s]












100%|██████████| 11/11 [00:

In [62]:
samples = np.abs(samples)
benchmark = []
for i in range(len(dU_error)):
    benchmark.append((np.mean(samples[i]),np.std(samples[i])))
print(benchmark)

[(10.278556092713476, 5.469665285993476), (0.13920275155267453, 0.15856610086446615), (0.02095639750383711, 0.02110178613472963), (0.0012863239205060494, 0.0019114710369066757), (0.0006491669082038243, 0.0), (0.0006491669082038243, 0.0)]


In [63]:
benchmark = np.array(benchmark)
print('mean:',benchmark[:,0])
print('std:',benchmark[:,1])

mean: [1.02785561e+01 1.39202752e-01 2.09563975e-02 1.28632392e-03
 6.49166908e-04 6.49166908e-04]
std: [5.46966529e+00 1.58566101e-01 2.11017861e-02 1.91147104e-03
 0.00000000e+00 0.00000000e+00]
